In [53]:
# Carga de librerias
# ------------------
# Librerias de uso general
import holidays
from google.cloud import storage

# Manejo de datos
import numpy as np
import pandas as pd

import kagglehub
import shutil
import os

import optuna
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM  # Importación para OC-SVM
from sklearn.neighbors import LocalOutlierFactor # Importación para LOF

from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from math import sqrt

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estadística y series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

# Se importan las funciones
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [54]:
column_mapper = {
    'Service': 'fecha',
    'Relational Database Service($)': 'rds',
    'EC2-Instances($)': 'ec2',
    'FSx($)': 'fsx',
    'Elastic File System($)': 'efs',
    'EC2-Other($)': 'ec2_other',
    'CloudWatch($)': 'cloudwatch',
    'Elastic Load Balancing($)': 'elb',
    'S3($)': 's3',
    'Backup($)': 'backup',
    'Key Management Service($)': 'kms',
    'DataSync($)': 'data_sync',
    'Secrets Manager($)': 'secrets_manager',
    'Resilience Hub($)': 'resiliency',
    'Total costs($)': 'total_costs'
}

Carga de los Datos
Se hace la carga de los datos desde la ruta Data 

In [55]:
# Datos con corte a Abril usar: "costs.csv"

df = pd.read_csv('/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/data/costs.csv')
df.head()

,Service,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),Total costs($)
0,Service total,34131.482733,23531.788153,5152.073356,2830.548352,2132.939335,1543.902136,778.604880,758.963353,530.170434,91.476443,34.06381,0.178930,0.0,71516.191916
1,2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0,55.206262
2,2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN,79.431926
3,2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN,158.866978
4,2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN,147.623435


In [56]:
# Mapeo de columnas 
df.rename(columns=column_mapper, inplace=True)
# Elimina la fila con índice 0 y ka columna total_cost
df = df.drop(0, axis=0)
df = df.drop('total_costs', axis=1)

# Prepocesar datos

Para nuestro caso de analisis vamos a aplicar una normalizacion

In [57]:
#Separar las variables continuas de las demás
df_continuas=df[["rds",
                       "ec2",
                       "fsx",
                       "efs",
                       "ec2_other",
                       "cloudwatch",
                       "s3",
                       "elb",
                       "backup",
                       "kms",
                       "data_sync",
                       "secrets_manager",
                       "resiliency"]]

In [58]:
# EL conjunto de datos se divide en una proporción de 60% (Entrenamiento), 20% (Validación) y 20% (Prueba).
X_train_val, X_test = train_test_split(df_continuas, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train_val, test_size=0.25, random_state=42) # 0.25 * 0.80 = 0.20 (20% para Validation)

In [59]:
# Normalizar los datos 
preprocessor_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor_pipeline.fit(X_train)

X_train_scaled = preprocessor_pipeline.transform(X_train)
X_val_scaled = preprocessor_pipeline.transform(X_val)
X_test_scaled = preprocessor_pipeline.transform(X_test)

# 4. Verificación y Persistencia (MLOps)
print(f"Media de 'rds' en X_train original: {X_train['rds'].mean():.4f}")
print(f"Media de 'rds' en X_train escalado: {X_train_scaled[:, X_train.columns.get_loc('rds')].mean():.4f}") # La media debe ser cercana a 0

# Guardar el Pipeline (¡Esto es clave para el MLOps!)
# Debes guardar el objeto preprocessor_pipeline para usarlo en producción
# cuando llegue un nuevo dato de vino.

OUTPUT_DIR = "./data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(os.path.join(OUTPUT_DIR, "preprocessor_pipeline.pkl"), "wb") as f:
    pickle.dump(preprocessor_pipeline, f)

print("-" * 50)
print(f"✅ Pipeline y datos escalados listos. El preprocesador se guardó en: {OUTPUT_DIR}")


Media de 'rds' en X_train original: 96.0687
Media de 'rds' en X_train escalado: -0.0000
--------------------------------------------------
✅ Pipeline y datos escalados listos. El preprocesador se guardó en: ./data/


In [60]:
# Crear los DF de nuevo con los datos ecaldos entregado en un array
# 1. Crear el DataFrame escalado
X_train_scaled_df = pd.DataFrame(
    # 1. Los datos son el array de NumPy
    X_train_scaled, 
    
    # 2. Los índices (filas) se toman del DataFrame original
    index=X_train.index, 
    
    # 3. Los nombres de las columnas se toman del DataFrame original
    columns=X_train.columns 
)

# 2. Repetir para los otros conjuntos
X_val_scaled_df = pd.DataFrame(
    X_val_scaled, 
    index=X_val.index, 
    columns=X_val.columns
)

X_test_scaled_df = pd.DataFrame(
    X_test_scaled, 
    index=X_test.index, 
    columns=X_test.columns
)

# 4. Verificación
print("\n--- Verificación del DataFrame escalado ---")
print(f"Tipo de X_train_scaled_df: {type(X_train_scaled_df)}")
print(f"Las primeras 5 filas de X_train_scaled_df:\n{X_train_scaled_df.head()}")
print(f"Media de 'rds' en X_train escalado (DF): {X_train_scaled_df['rds'].mean():.4f}")


--- Verificación del DataFrame escalado ---
Tipo de X_train_scaled_df: <class 'pandas.core.frame.DataFrame'>
Las primeras 5 filas de X_train_scaled_df:
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
175 -0.055722 -0.455170  0.662202 -0.666575   0.129742   -0.405917 -0.247247   
233 -0.069585  1.030785  0.492330  0.707702   0.541876   -0.471159  0.564653   
351  3.891164 -0.703529  0.508479  1.861979  -1.265586   -0.166167  1.434175   
75  -0.648316 -0.494546  0.333014 -0.940589   0.849970   -0.330212 -0.945479   
66  -0.665479 -0.702253 -1.098427 -0.944476  -0.040083   -0.472826 -1.103862   

          elb    backup       kms  data_sync  secrets_manager  resiliency  
175 -0.408174 -0.575719  0.224661  -0.111943        -0.539126         0.0  
233 -0.459486  0.489771 -0.058055  -0.111944        -0.140724         0.0  
351 -0.116118 -0.381946  2.932068  -0.111943        -0.695362         0.0  
75  -0.328749 -0.995102 -1.013494  -0.111943         0.062383 

Isolation Forest (El Enfoque Robusto)
El Isolation Forest es rápido, escalable y muy robusto para datos multivariados con muchos outliers.

In [61]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_is = X_train_scaled_df.copy() 
# creamos una columna de etiquetas verdaderas *para el ejemplo*
# En la vida real, necesitarías cargar estas etiquetas.
# Ejemplo de etiquetas verdaderas simuladas:
# 99% normales (1), 1% anómalas (-1)
contamination_factor = 0.01
X_train_size = X_train_scaled_df_is.shape[0]
n_anomalies = int(X_train_size * 0.01)
y_true = np.ones(X_train_size)
y_true[:n_anomalies] = -1 # Los primeros datos son anómalos
np.random.shuffle(y_true) # Mezclamos las etiquetas
X_train_scaled_df_is['is_anomaly_true'] = y_true # Agregamos la columna de etiquetas verdaderas


# Inicializar y entrenar el modelo
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=contamination_factor,
    random_state=42,
    n_jobs=-1
)

X_train_features = X_train_scaled_df_is.copy() 

# Entrenar el modelo (No supervisado: no necesita etiquetas)
iso_forest.fit(X_train_features.drop(columns=['is_anomaly_true'])) # Entrenamos sin la etiqueta verdadera

# Predecir las anomalías: -1 para anomalía, 1 para dato normal
X_train_scaled_df_is['anomaly_iso_forest'] = iso_forest.predict(X_train_features.drop(columns=['is_anomaly_true']))

# Obtener la 'Puntuación de Anomalía'
X_train_scaled_df_is['score_iso_forest'] = iso_forest.decision_function(X_train_features.drop(columns=['is_anomaly_true']))

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por Isolation Forest (ejemplo) ---")
anomalies_if = X_train_scaled_df_is[X_train_scaled_df_is['anomaly_iso_forest'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_if)}")
print(anomalies_if.head())
# Nota: El umbral para la detección es ajustado por el 'contamination' que elegiste.

# --- 2. CÁLCULO DE MÉTRICAS DE DESEMPEÑO ---

# a) Métricas de Clasificación (F1-Score)
# Necesitan las etiquetas verdaderas (y_true) y las etiquetas predichas (y_pred)

y_pred = X_train_scaled_df_is['anomaly_iso_forest'].values
y_true = X_train_scaled_df_is['is_anomaly_true'].values

# El F1-Score es la media armónica entre precisión y exhaustividad.
# Es ideal cuando las clases están desbalanceadas (muchos normales, pocos anómalos).
f1 = f1_score(y_true, y_pred, average='binary', pos_label=-1) 

print(f"F1-Score (enfocado en anomalías): {f1:.4f}")

# b) Métrica de Error (RMSE)
# El RMSE se usa típicamente para modelos de Regresión, pero se puede usar para medir
# qué tan lejos están las *puntuaciones* de anomalía de un valor objetivo.

# **ADVERTENCIA sobre RMSE en Detección de Anomalías:**
# El RMSE no es una métrica estándar para evaluar el rendimiento de clasificación
# de Isolation Forest. No obstante, si se quiere forzar la métrica:

# Creamos una 'puntuación ideal' donde los valores normales tienen 1.0 y los anómalos tienen un valor bajo (ej. 0.0)
# Para el ejemplo, usaremos la puntuación de decisión (score_iso_forest)
# La forma correcta de usar RMSE aquí es compleja, así que mostraremos cómo calcularlo
# si tu objetivo fuese predecir la puntuación verdadera.

# Para simplificar y mostrar el cálculo:
# Asumamos que tu objetivo (y_true) son las etiquetas numéricas, y queremos ver
# el "error" de la puntuación de decisión.
# (Esto no es un uso estándar, pero cumple con el requisito de mostrar el cálculo)
# Vamos a simular un 'score' objetivo: 1 para normal, 0 para anomalía
y_target_score = np.where(y_true == 1, 1.0, 0.0) 
y_pred_score = X_train_scaled_df_is['score_iso_forest'].values # Las puntuaciones de decisión

rmse_score = sqrt(mean_squared_error(y_target_score, y_pred_score))

print(f"RMSE (entre puntuación ideal y decisión): {rmse_score:.4f}")


# --- 3. RESUMEN DE MÉTRICAS ---

# Puedes agrupar las métricas en un diccionario para un informe fácil
performance_metrics = {
    'F1_Score': f1,
    'RMSE_Score': rmse_score,
    # Otras métricas comunes en anomalías:
    # 'Precision': precision_score(y_true, y_pred, pos_label=-1),
    # 'Recall': recall_score(y_true, y_pred, pos_label=-1)
}

print("\n--- RESUMEN DE DESEMPEÑO ---")
for metric, value in performance_metrics.items():
    print(f"{metric}: {value:.4f}")


--- Anomalías detectadas por Isolation Forest (ejemplo) ---
Total de anomalías detectadas: 3
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
351  3.891164 -0.703529  0.508479  1.861979  -1.265586   -0.166167  1.434175   
349  3.902792  0.535344  0.772158  1.861837   0.704645    1.506159  1.424562   
344  3.171395 -0.539961  0.593477  1.861602  -0.722283   -0.178437  1.426032   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
351 -0.116118 -0.381946  2.932068  -0.111943        -0.695362         0.0   
349  6.726628  2.150503  2.939841  -0.111943         8.569433         0.0   
344  3.981615  2.150536  2.931883  -0.111943        -0.539126         0.0   

     is_anomaly_true  anomaly_iso_forest  score_iso_forest  
351              1.0                  -1         -0.022894  
349              1.0                  -1         -0.060376  
344              1.0                  -1         -0.000441  
F1-Score (enfocado en anomalías)

One-Class SVM (OC-SVM)
OC-SVM intenta encontrar un hiperplano que separe la mayoría de los datos "normales" del origen, marcando los puntos que quedan fuera como anomalías.

In [66]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_oc = X_train_scaled_df.copy() 

# --- CONFIGURACIÓN DEL MODELO ---

# El parámetro 'nu' es el equivalente al 'contamination' factor,
# representando la fracción superior de anomalías esperadas.
# 'gamma' define la influencia de un punto de entrenamiento individual
# (usamos 'auto' para que se calibre por 1/n_features).
contamination_factor = 0.01  # Esperamos un 1% de anomalías
n_samples = 210
n_features = 13

# 2. CREAR ETIQUETAS VERDADERAS DE EJEMPLO (y_true_oc_svm)
# Asumimos que el 2% de los datos son anomalías (-1).
y_true_oc_svm = np.ones(n_samples, dtype=int)
y_true_oc_svm[np.random.choice(n_samples, size=int(n_samples * 0.02), replace=False)] = -1
y_true_oc_svm = pd.Series(y_true_oc_svm, name='is_anomaly_true')

# Inicializar el modelo OC-SVM
oc_svm = OneClassSVM(
    kernel='rbf',  # Kernel de función de base radial, común en anomalías
    gamma='auto',
    nu=contamination_factor
)

X_train_features = X_train_scaled_df_is.copy() 

# Entrenar el modelo
oc_svm.fit(X_train_features)

# Predecir las anomalías: -1 para anomalía, 1 para dato normal
X_train_scaled_df_oc['anomaly_oc_svm'] = oc_svm.predict(X_train_features)

# Obtener la 'Puntuación de Anomalía' (cuanto menor, más anómalo)
X_train_scaled_df_oc['score_oc_svm'] = oc_svm.decision_function(X_train_features)

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por One-Class SVM ---")
anomalies_ocsvm = X_train_scaled_df_oc[X_train_scaled_df_oc['anomaly_oc_svm'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_ocsvm)}")
print(anomalies_ocsvm.sort_values(by='score_oc_svm').head())

# --- CÁLCULO DE MÉTRICAS ROBUSTAS AL DESEQUILIBRIO ---

# 1. Etiquetas y Predicciones
# Aquí debes asegurarte de que tu 'y_true' esté cargado y alineado con X_train_features
y_true = y_true_oc_svm # Usamos la variable de etiquetas verdaderas
y_pred = X_train_scaled_df_oc['anomaly_oc_svm']
y_score = X_train_scaled_df_oc['score_oc_svm'] # Usamos el score para el ROC AUC

# 2. Métricas de Clasificación (Dependientes del Contamination Factor)
# Establecemos pos_label=-1 para enfocarnos en la clase minoritaria (Anomalía)
f1 = f1_score(y_true, y_pred, pos_label=-1)
precision = precision_score(y_true, y_pred, pos_label=-1)
recall = recall_score(y_true, y_pred, pos_label=-1)

# 3. Métrica de Curva (Independiente del Umbral)
# Transformamos y_true a binario: 1 si es anomalía (-1), 0 si es normal (1).
y_true_binary = np.where(y_true == -1, 1, 0) 
# Nota: En OC-SVM, 'score_oc_svm' es la distancia al hiperplano (menor score = más anómalo).
# Para AUC, necesitamos que un score más alto corresponda a la clase positiva (1, la anomalía).
# Por lo tanto, invertimos la puntuación: un score más bajo se convierte en un valor alto.
y_score_auc = -y_score

try:
    # Usamos la puntuación invertida y la etiqueta binaria
    roc_auc = roc_auc_score(y_true_binary, y_score_auc)
except ValueError as e:
    # Capturamos el error si no hay suficientes ejemplos de ambas clases
    roc_auc = "N/A (Se necesita al menos una muestra de ambas clases)"


# --- IMPRESIÓN DE RESULTADOS ---

print("\n--- Evaluación del Modelo One-Class SVM ---")
print(f"Factor de Contaminación Usado (nu): {contamination_factor*100:.2f}%")
print("-" * 40)
print(f"F1-Score (Equilibrio Precision/Recall): {f1:.4f}")
print(f"Precision (Alerta Verdadera): {precision:.4f}")
print(f"Recall/Exhaustividad (Anomalía Encontrada): {recall:.4f}")
print("-" * 40)
print(f"ROC AUC (Rendimiento General, Sin Umbral): {roc_auc:.4f}")

# Mostrar los días anómalos
print("\n--- Detalles de Anomalías Detectadas (OC-SVM) ---")
anomalies_ocsvm = X_train_scaled_df_oc[X_train_scaled_df_oc['anomaly_oc_svm'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_ocsvm)}")
print(anomalies_ocsvm.sort_values(by='score_oc_svm').head())


--- Anomalías detectadas por One-Class SVM ---
Total de anomalías detectadas: 17
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
288  0.328941  0.889130  0.754669  1.041624  -0.565684   -0.471476  1.184117   
323  1.836257  0.893155  1.078494  1.936324   1.628101    2.352258  1.486355   
203 -0.328424  0.436305  0.450120 -0.129187   0.464148   -0.462137 -0.085947   
350  3.951215  0.488619  0.735016  1.861917   0.586157    1.324748  1.417203   
92  -0.883599 -3.041664 -1.334212 -0.924425  -1.835021   -0.546203 -0.449326   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
288 -0.456472  1.953022 -0.051207   0.613122        -0.023547         0.0   
323 -0.457923  1.644362  2.269787  -0.111943        -0.226654         0.0   
203  0.263859 -0.138673 -0.053058   9.541929        -0.671926         0.0   
350  3.381542  2.150591  2.944005  -0.111943        -0.445384         0.0   
92  -0.461212 -0.980531 -1.051246  -0.111943        

Local Outlier Factor (LOF) LOF es un modelo basado en la densidad. Mide cuán aislado está un punto con respecto a sus vecinos más cercanos. Un score LOF mucho mayor que 1 indica una anomalía.

In [ ]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
X_train_scaled_df_lof = X_train_scaled_df.copy() 
X_train_scaled_df_lof['is_anomaly_true'] = np.where(
    np.random.rand(len(X_train_scaled_df_lof)) < 0.01, -1, 1
)

# --- CONFIGURACIÓN DEL MODELO ---

# 'n_neighbors': Número de vecinos a considerar (típicamente entre 20 y 100).
# 'contamination': La proporción esperada de anomalías (similar a Isolation Forest).
n_neighbors = 20
contamination_factor = 0.01

# Inicializar el modelo LOF
# Nota: LOF se inicializa en 'novelty=False' para fines de predicción.
lof = LocalOutlierFactor(
    n_neighbors=n_neighbors,
    contamination=contamination_factor,
    novelty=False  # Para usarlo en modo detección de outliers en el conjunto de entrenamiento
)

# Ajustar y predecir (LOF realiza el ajuste y la predicción en un solo paso)
X_train_scaled_df_lof['anomaly_lof'] = lof.fit_predict(X_train_scaled_df_lof.drop(columns=['is_anomaly_true']))

# Puntuación de Anomalía
X_train_scaled_df_lof['score_lof'] = -lof.negative_outlier_factor_

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por Local Outlier Factor (LOF) ---")
anomalies_lof = X_train_scaled_df_lof[X_train_scaled_df_lof['anomaly_lof'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_lof)}")
print(anomalies_lof.sort_values(by='score_lof', ascending=False).head())

# --- CÁLCULO DE MÉTRICAS ROBUSTAS AL DESEQUILIBRIO ---

# 1. Etiquetas y Predicciones
y_true = X_train_scaled_df_lof['is_anomaly_true']
y_pred = X_train_scaled_df_lof['anomaly_lof']
y_score = X_train_scaled_df_lof['score_lof'] # Usamos el score para el ROC AUC

# 2. Métricas de Clasificación (Dependientes del Contamination Factor)
# Establecemos pos_label=-1 para enfocarnos en la clase minoritaria (Anomalía)
f1 = f1_score(y_true, y_pred, pos_label=-1)
precision = precision_score(y_true, y_pred, pos_label=-1)
recall = recall_score(y_true, y_pred, pos_label=-1)

# 3. Métrica de Curva (Independiente del Umbral)
# Nota: ROC AUC requiere que la clase positiva sea 1 y la negativa 0. 
# Transformamos -1 (Anomalía) -> 0 y 1 (Normal) -> 1 para las etiquetas verdaderas.
# Usamos el score para la curva.
y_true_binary = np.where(y_true == 1, 0, 1) # 1 si es anomalía, 0 si es normal
# Nota: En LOF, score_lof (nuestro valor) es más alto para la anomalía, lo cual es correcto para AUC.

try:
    roc_auc = roc_auc_score(y_true_binary, y_score)
except ValueError as e:
    roc_auc = "N/A (Se necesita al menos una muestra de ambas clases)"


# --- IMPRESIÓN DE RESULTADOS ---

print("\n--- Evaluación del Modelo LOF ---")
print(f"Factor de Contaminación Usado: {contamination_factor*100}%")
print("-" * 40)
print(f"F1-Score (Equilibrio Precision/Recall): {f1:.4f}")
print(f"Precision (Alerta Verdadera): {precision:.4f}")
print(f"Recall/Exhaustividad (Anomalía Encontrada): {recall:.4f}")
print("-" * 40)
print(f"ROC AUC (Rendimiento General, Sin Umbral): {roc_auc}")


--- Anomalías detectadas por Local Outlier Factor (LOF) ---
Total de anomalías detectadas: 3
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
203 -0.328424  0.436305  0.450120 -0.129187   0.464148   -0.462137 -0.085947   
224 -0.250073  1.075655  0.459695  0.194370   0.590359   -0.474813  0.177261   
252  0.036355  1.020134  1.813887  0.904283   0.227462   -0.380775  1.060813   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
203  0.263859 -0.138673 -0.053058   9.541929        -0.671926         0.0   
224 -0.045358  0.028645  0.061032  -0.111942         8.460067         0.0   
252  0.256140  2.096955  0.797905  10.679210        -0.086041         0.0   

     is_anomaly_true  anomaly_lof  score_lof  
203                1           -1   6.819289  
224                1           -1   6.034158  
252                1           -1   5.322369  

--- Evaluación del Modelo LOF ---
Factor de Contaminación Usado: 1.0%
-----------------

##Encontrar los mejores Hiperparametros con Optuna

In [ ]:
# Usaremos X_train_scaled_df para la validación cruzada.
X_train_data = X_train_scaled_df.copy() # Usaremos este para entrenamiento y validación interna

# --- FUNCIÓN OBJETIVO DE OPTUNA ---
# Optuna llamará a esta función repetidamente con diferentes hiperparámetros.
# El objetivo es minimizar (o maximizar) el valor retornado (trial.suggest).

def objective(trial, X_data):
    """
    Define el modelo, sugiere hiperparámetros y retorna una métrica de rendimiento.
    """
    
    # 1. Sugerencia de Hiperparámetros
    # a) Kernel
    kernel = trial.suggest_categorical('kernel', ['rbf', 'sigmoid'])
    
    # b) Gamma (solo si el kernel es 'rbf' o 'sigmoid')
    if kernel in ['rbf', 'sigmoid']:
        gamma = trial.suggest_loguniform('gamma', 1e-3, 1e0) # Rango de 0.001 a 1.0
    else:
        gamma = 'auto' # o 'scale'
        
    # c) Nu (Factor de contaminación)
    # Rango típico de 0.001 a 0.1 (0.1% a 10% de anomalías)
    nu = trial.suggest_loguniform('nu', 1e-3, 1e-1) 
    
    # 2. Inicialización del Modelo
    model = OneClassSVM(
        kernel=kernel,
        gamma=gamma,
        nu=nu
    )
    
    # 3. Evaluación del Modelo
    
    # A) MÉTRICA SUPERVISADA (Si tuvieras etiquetas 'y_train_true'):
    # Si tuvieras y_train_true, usarías F1-Score:
    # f1_scorer = make_scorer(f1_score, pos_label=-1)
    # cv = KFold(n_splits=5, shuffle=True, random_state=42)
    # score = cross_val_score(model, X_data, y_train_true, scoring=f1_scorer, cv=cv, n_jobs=-1).mean()
    # return score

    # B) MÉTRICA NO SUPERVISADA (Lo más probable para OC-SVM):
    # En detección de anomalías no supervisada, es difícil usar CV estándar.
    # El método más robusto es entrenar en Train y evaluar la densidad/score en Validation (X_val_scaled_df).
    
    # Entrenar en todo el conjunto de entrenamiento disponible
    model.fit(X_data) 
    
    # Usamos la Media de la función de decisión (Score) en el conjunto de *validación* (X_val_scaled_df).
    # Un score más alto en el set de validación (asumiendo que es mayormente normal) es generalmente mejor.
    # Opcionalmente, se podría usar la Tasa de Detección (si se asume un nu)
    
    # **Usaremos una métrica simple para optimización de OC-SVM:**
    # La media de la puntuación de decisión en el conjunto de entrenamiento.
    # Esto es heurístico, pero busca la frontera que maximice la "normalidad" interna.
    decision_scores = model.decision_function(X_data)
    
    # Optuna busca MINIMIZAR por defecto, por lo que retornamos el negativo de la media
    # para MAXIMIZAR la media de la función de decisión.
    # Se podría usar la media del conjunto de VALIDACIÓN (X_val_scaled_df) si lo tienes.
    avg_score = -np.mean(decision_scores) # Queremos maximizar la media, así que minimizamos su negativo.
    
    return avg_score

# --- EJECUCIÓN DE OPTUNA ---

# 1. Crear el estudio
# direction='minimize' (porque estamos minimizando -np.mean(decision_scores))
study = optuna.create_study(direction='minimize', study_name='oc_svm_optimization')

# 2. Iniciar la optimización
# n_trials: Número de combinaciones de hiperparámetros a probar.
study.optimize(lambda trial: objective(trial, X_train_data), n_trials=50, n_jobs=-1)

# 3. Mostrar los resultados
print("\n--- Resultados de la Optimización con Optuna ---")
print(f"Mejor prueba (Trial): {study.best_trial.number}")
print(f"Mejor valor (Puntuación media NEGATIVA): {study.best_value:.4f}")
print(f"Mejores Hiperparámetros: {study.best_params}")

# --- REENTRENAR EL MODELO FINAL CON LOS MEJORES HIPERPARÁMETROS ---

best_params = study.best_params

final_oc_svm = OneClassSVM(
    kernel=best_params['kernel'],
    gamma=best_params.get('gamma', 'auto'), # Usar .get() por si 'gamma' no se definió
    nu=best_params['nu']
)

final_oc_svm.fit(X_train_data)

# Finalmente, evaluamos el modelo final en el conjunto de prueba (X_test_scaled_df) si está disponible.

[I 2025-10-26 16:12:49,863] A new study created in memory with name: oc_svm_optimization
/var/folders/nt/4ldzwbz51gg979lps65n9sjr0000gn/T/ipykernel_46907/2493189873.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-3, 1e0) # Rango de 0.001 a 1.0
/var/folders/nt/4ldzwbz51gg979lps65n9sjr0000gn/T/ipykernel_46907/2493189873.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  nu = trial.suggest_loguniform('nu', 1e-3, 1e-1)
/var/folders/nt/4ldzwbz51gg979lps65n9sjr0000gn/T/ipykernel_46907/2493189873.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.


--- Resultados de la Optimización con Optuna ---
Mejor prueba (Trial): 49
Mejor valor (Puntuación media NEGATIVA): -5.4688
Mejores Hiperparámetros: {'kernel': 'sigmoid', 'gamma': 0.6538409563410645, 'nu': 0.096748867587444}


,kernel,'sigmoid'
,degree,3
,gamma,0.6538409563410645
,coef0,0.0
,tol,0.001
,nu,0.096748867587444
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


## Evaluamos el modelo con los mejores parametro y el datafame de test

In [ ]:
# Usaremos X_train_scaled_df para la validación cruzada.
X_train_data = X_train_scaled_df.copy() # Usaremos este para entrenamiento y validación interna

# --- 1. DEFINICIÓN DE LOS MEJORES PARÁMETROS
best_params_ocsvm = {
    'kernel': 'sigmoid',
    # Para One-Class SVM en sklearn, 'gamma' y 'nu' son float.
    'gamma': 0.9433749497213476,
    'nu': 0.0975932191542979,
    # Los siguientes son los valores por defecto que se ven en la imagen (pueden omitirse)
    'degree': 3,
    'coef0': 0.0,
    'tol': 0.001,
    'shrinking': True,
    'cache_size': 200,
    'verbose': False,
    'max_iter': -1
}

# --- 2. INSTANCIAR Y ENTRENAR EL MODELO FINAL ---

print("Iniciando entrenamiento del modelo final OC-SVM...")

# Creamos el modelo usando solo los hiperparámetros clave (kernel, gamma, nu)
final_oc_svm = OneClassSVM(
    kernel=best_params_ocsvm['kernel'],
    gamma=best_params_ocsvm['gamma'],
    nu=best_params_ocsvm['nu'],
    # Otros parámetros opcionales
    coef0=best_params_ocsvm['coef0'],
    tol=best_params_ocsvm['tol']
)

# Entrenar el modelo con los mejores parámetros sobre el conjunto de entrenamiento/validación
final_oc_svm.fit(X_train_data)

print("Entrenamiento completado.")

# --- 3. EVALUACIÓN EN EL CONJUNTO DE PRUEBA (X_test_scaled_df) ---

print("Evaluando el modelo en el conjunto de prueba (X_test_scaled_df)...")

# a) Predicción de Clase (-1: Anomalía, 1: Normal)
y_pred_test = final_oc_svm.predict(X_test_scaled_df)

# b) Puntuación de Decisión (Decision Function Score)
# Esta es la distancia firmada del punto al hiperplano separador. 
# Valores negativos grandes indican mayor anomalía.
score_test = final_oc_svm.decision_function(X_test_scaled_df)

# c) Asignar los resultados de vuelta al DataFrame de prueba (si es un DataFrame)
if isinstance(X_test_scaled_df, pd.DataFrame):
    X_test_scaled_df['oc_svm_prediction'] = y_pred_test
    X_test_scaled_df['oc_svm_score'] = score_test
    
    # Mostrar las 5 anomalías más severas en el conjunto de prueba
    anomalies_test = X_test_scaled_df[X_test_scaled_df['oc_svm_prediction'] == -1]
    
    print(f"\nTotal de anomalías detectadas en Test: {len(anomalies_test)}")
    print("\nTop 5 anomalías más severas (menor score):")
    print(anomalies_test.sort_values(by='oc_svm_score').head())

else:
    print("\nResultados generados como arrays. Asegúrese de unirlos al DataFrame original para su análisis.")

Iniciando entrenamiento del modelo final OC-SVM...
Entrenamiento completado.
Evaluando el modelo en el conjunto de prueba (X_test_scaled_df)...

Total de anomalías detectadas en Test: 9

Top 5 anomalías más severas (menor score):
          rds       ec2       fsx       efs  ec2_other  cloudwatch        s3  \
186 -0.383677  0.712471  0.425781 -0.195942   0.342699   -0.472680 -0.100148   
343  3.066193  0.485987  0.689289  1.732137   0.675306    1.275004  1.428934   
78  -0.872591 -3.041149 -1.335344 -0.940244  -1.839802   -0.546347 -0.911080   
127 -0.856453 -3.037265 -1.332167 -0.758057  -1.838779   -0.546592 -0.490228   
85  -0.884482 -3.041005 -1.336444 -0.926049  -1.839498   -0.546795 -0.755983   

          elb    backup       kms  data_sync  secrets_manager  resiliency  \
186 -0.097256 -0.296151 -0.054076   9.325677        -0.664115         0.0   
343  3.938539  1.770747  2.939656  -0.111943        -0.695362         0.0   
78  -0.359672 -0.994568 -1.032370  -0.111943         0.226